In [1]:
!pip install --upgrade ibm-watsonx-ai ibm-cos-sdk pandas numpy scikit-learn nltk transformers
!pip install --upgrade "ibm-granular-access-sdk[iam-token-manager]" # For IBM Granite (if you're using it directly with API)

import nltk
nltk.download('punkt')
nltk.download('stopwords')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 8.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 31.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 29.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 42.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 112.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.6/139.6 kB 48.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 148.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 40.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 154.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 14

[nltk_data] Downloading package punkt to /home/wsuser/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /home/wsuser/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [2]:

api_key = "y0NlbXIBHDsEhgbRnYuDvatJFJDqv9yt98Uj0WmAUtJA"
project_id = "b2001e10-b6ff-497e-9afa-adc1eee2b43c"
watsonx_ai_url = "https://au-syd.ml.cloud.ibm.com"


cos_endpoint = "https://s3.au-syd.cloud-object-storage.appdomain.cloud"
cos_service_crn = "crn:v1:bluemix:public:cloud-object-storage:global:a/df3e666bca344941b4d3e0c68d38d187:4041aeb9-d4c6-40fa-96da-991bcfa82493::"


from ibm_watsonx_ai.client import APIClient

try:
    credentials = {
        "api_key": api_key,
        "url": watsonx_ai_url 
    }
    client = APIClient(credentials=credentials, project_id=project_id)
    print("Watsonx.ai client initialized successfully!")
except Exception as e:
    print(f"Error initializing Watsonx.ai client: {e}")
    print("Please check your API key, project ID, and URL.")

import ibm_boto3
from ibm_botocore.client import Config, ClientError

try:
    cos_client = ibm_boto3.client(
        service_name='s3',
        ibm_api_key_id=api_key,
        ibm_service_instance_id=cos_service_crn,
        config=Config(signature_version='oauth'),
        endpoint_url=cos_endpoint
    )
    print("IBM Cloud Object Storage client initialized successfully!")
except Exception as e:
    print(f"Error initializing COS client: {e}")
    print("Please ensure your COS endpoint and service CRN are correct and accessible.")

Watsonx.ai client initialized successfully!
IBM Cloud Object Storage client initialized successfully!


In [3]:

print("Listing COS buckets...")
try:
    response = cos_client.list_buckets()
    for bucket in response['Buckets']:
        print(f"  Bucket Name: {bucket['Name']}")
except Exception as e:
    print(f"Error listing buckets: {e}")

Listing COS buckets...
  Bucket Name: aidrivenplagiarismintelligencefor-donotdelete-pr-ktjvnitkzeqpim


In [4]:

target_bucket_name = "aidrivenplagiarismintelligencefor-donotdelete-pr-ktjvnitkzeqpim" 

print(f"\nListing objects in bucket: {target_bucket_name}")
try:
    response = cos_client.list_objects(Bucket=target_bucket_name)
    if 'Contents' in response:
        for obj in response['Contents']:
            print(f"  Object Key: {obj['Key']} (Size: {obj['Size']} bytes)")
    else:
        print(f"No objects found in bucket: {target_bucket_name}")
except Exception as e:
    print(f"Error listing objects in bucket {target_bucket_name}: {e}")


Listing objects in bucket: aidrivenplagiarismintelligencefor-donotdelete-pr-ktjvnitkzeqpim
  Object Key: notebook/GranitePlagiarismChecker_zwC31Uwi4.ipynb (Size: 30864 bytes)
  Object Key: ref1.txt (Size: 192 bytes)
  Object Key: ref2.txt (Size: 186 bytes)
  Object Key: submissions/GranitePlagiarismChecker_zwC31Uwi4.ipynb (Size: 20440 bytes)
  Object Key: user.txt (Size: 255 bytes)


In [5]:
import os

bucket_name = "aidrivenplagiarismintelligencefor-donotdelete-pr-ktjvnitkzeqpim"
object_key = "notebook/GranitePlagiarismChecker_zwC31Uwi4.ipynb"
local_file_name = "GranitePlagiarismChecker_zwC31Uwi4.ipynb" # Name to save it as locally in the notebook environment

print(f"Downloading {object_key} from bucket {bucket_name}...")
try:
    cos_client.download_file(bucket_name, object_key, local_file_name)
    print(f"Downloaded {object_key} to {local_file_name}")
   
except Exception as e:
    print(f"Error downloading {object_key}: {e}")

Downloaded notebook/GranitePlagiarismChecker_zwC31Uwi4.ipynb to GranitePlagiarismChecker_zwC31Uwi4.ipynb


In [6]:
from ibm_boto3 import client
from ibm_botocore.client import Config


assignment_file = "GranitePlagiarismChecker_zwC31Uwi4.ipynb"
cos_client.upload_file(Filename=assignment_file,
                       Bucket=bucket_name,
                       Key=f"submissions/{assignment_file}")

print(f"✅ Uploaded {assignment_file} to submissions folder in bucket {bucket_name}")


✅ Uploaded GranitePlagiarismChecker_zwC31Uwi4.ipynb to submissions folder in bucket aidrivenplagiarismintelligencefor-donotdelete-pr-ktjvnitkzeqpim


In [7]:
from ibm_watsonx_ai.foundation_models import ModelInference
from ibm_watsonx_ai.client import APIClient


api_key = "y0NlbXIBHDsEhgbRnYuDvatJFJDqv9yt98Uj0WmAUtJA"
project_id = "b2001e10-b6ff-497e-9afa-adc1eee2b43c"
watsonx_ai_url = "https://au-syd.ml.cloud.ibm.com"


credentials = {
    "url": watsonx_ai_url,
    "api_key": api_key
}

client = APIClient(credentials=credentials, project_id=project_id)
print("✅ Watsonx API client initialized successfully.")


model = ModelInference(
    model_id="ibm/granite-13b-instruct-v2",
    credentials=credentials,
    project_id=project_id
)
print("✅ Granite model loaded successfully.")

prompt = "Explain the difference between supervised and unsupervised learning in simple terms."

parameters = {
    "decoding_method": "sample",
    "max_new_tokens": 100,
    "temperature": 0.5,
    "top_k": 50,
    "top_p": 1
}

response = model.generate(prompt=prompt, params=parameters)

print("\n🧠 Model Response:\n")
print(response)


✅ Watsonx API client initialized successfully.


/opt/conda/envs/Python-RT24.1/lib/python3.11/site-packages/ibm_watsonx_ai/foundation_models/utils/utils.py:436: LifecycleWarning: Model 'ibm/granite-13b-instruct-v2' is in deprecated state from 2025-06-18 until 2025-10-15. IDs of alternative models: ibm/granite-3-3-8b-instruct. Further details: https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/fm-model-lifecycle.html?context=wx&audience=wdp
  warn(model_state_warning, category=LifecycleWarning)


✅ Granite model loaded successfully.

🧠 Model Response:

{'model_id': 'ibm/granite-13b-instruct-v2', 'created_at': '2025-08-02T18:51:14.301Z', 'results': [{'generated_text': 'Supervised learning is a type of machine learning that uses labeled data to learn a function that maps an input to an output. Unsupervised learning is a type of machine learning that does not use labeled data to learn a function that maps an input to an output. ', 'generated_token_count': 54, 'input_token_count': 14, 'stop_reason': 'eos_token', 'seed': 2760205612}], 'system': {'warnings': [{'message': "Model 'ibm/granite-13b-instruct-v2' is in deprecated state from 2025-06-18. It will be in withdrawn state from 2025-10-15. IDs of alternative models: ibm/granite-3-3-8b-instruct.", 'id': 'deprecation_warning', 'more_info': 'https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/fm-model-lifecycle.html?context=wx&audience=wdp'}, {'message': 'In future implementation, the parameter `parameters.decoding_metho

In [8]:
with open("user.txt", "w") as f:
    f.write("Machine learning is a field of AI that enables computers to learn from data without being explicitly programmed.")

with open("ref1.txt", "w") as f:
    f.write("Machine learning enables systems to learn and improve from experience without being explicitly programmed.")

with open("ref2.txt", "w") as f:
    f.write("Artificial Intelligence encompasses machine learning, where data-driven algorithms learn and make predictions.")


In [10]:
from ibm_watsonx_ai.foundation_models import ModelInference
import json

# Step 1: Load the files
def read_file(filename):
    with open(filename, "r", encoding="utf-8") as f:
        return f.read()

# Update filenames if your uploaded files are named differently
user_text = read_file("user.txt")
ref1_text = read_file("ref1.txt")
ref2_text = read_file("ref2.txt")

# Step 2: Initialize the model
model = ModelInference(
    model_id="ibm/granite-13b-instruct-v2",  
    credentials={
        "api_key": "y0NlbXIBHDsEhgbRnYuDvatJFJDqv9yt98Uj0WmAUtJA",
        "url": "https://au-syd.ml.cloud.ibm.com"
    },
    project_id="b2001e10-b6ff-497e-9afa-adc1eee2b43c"
)
print("✅ Model Initialized")

# Step 3: Create prompt
prompt = f"""Check this assignment for plagiarism.
Compare it with the reference materials below.

User Submission:
{user_text}

Reference 1:
{ref1_text}

Reference 2:
{ref2_text}
"""

# Step 4: Generate and print the result
response = model.generate(prompt=prompt)

try:
    if isinstance(response, str):
        response = json.loads(response)

    print("\n🧠 Plagiarism Detection Report:\n")
    print(response['results'][0]['generated_text'])

    with open("plagiarism_report.txt", "w") as f:
        f.write(response['results'][0]['generated_text'])
    print("✅ Report saved as plagiarism_report.txt")

except Exception as e:
    print("⚠️ Failed to parse response.")
    print("Raw output:\n", response)
    print("\nError:\n", e)


/opt/conda/envs/Python-RT24.1/lib/python3.11/site-packages/ibm_watsonx_ai/foundation_models/utils/utils.py:436: LifecycleWarning: Model 'ibm/granite-13b-instruct-v2' is in deprecated state from 2025-06-18 until 2025-10-15. IDs of alternative models: ibm/granite-3-3-8b-instruct. Further details: https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/fm-model-lifecycle.html?context=wx&audience=wdp
  warn(model_state_warning, category=LifecycleWarning)


✅ Model Initialized

🧠 Plagiarism Detection Report:


Yes
✅ Report saved as plagiarism_report.txt


In [11]:
# --- Setup: Download files from IBM COS ---
import ibm_boto3
from ibm_botocore.client import Config
from ibm_watsonx_ai.foundation_models import ModelInference
import json

# COS credentials
cos_endpoint = "https://s3.au-syd.cloud-object-storage.appdomain.cloud"
cos_api_key = "y0NlbXIBHDsEhgbRnYuDvatJFJDqv9yt98Uj0WmAUtJA"
cos_resource_crn = "crn:v1:bluemix:public:cloud-object-storage:global:a/df3e666bca344941b4d3e0c68d38d187:4041aeb9-d4c6-40fa-96da-991bcfa82493::"
bucket_name = "aidrivenplagiarismintelligencefor-donotdelete-pr-ktjvnitkzeqpim"

cos = ibm_boto3.client(
    "s3",
    ibm_api_key_id=cos_api_key,
    ibm_service_instance_id=cos_resource_crn,
    config=Config(signature_version="oauth"),
    endpoint_url=cos_endpoint
)

for file in ["user.txt", "ref1.txt", "ref2.txt"]:
    cos.download_file(Bucket=bucket_name, Key=file, Filename=file)
    print(f"✅ Downloaded: {file}")

# --- Read files ---
def read_file(filename):
    with open(filename, "r", encoding="utf-8") as f:
        return f.read()

user_text = read_file("user.txt")
ref1_text = read_file("ref1.txt")
ref2_text = read_file("ref2.txt")

# --- Initialize the Model ---
model = ModelInference(
    model_id="ibm/granite-13b-instruct-v2",
    credentials={
        "api_key": cos_api_key,
        "url": "https://au-syd.ml.cloud.ibm.com"
    },
    project_id="b2001e10-b6ff-497e-9afa-adc1eee2b43c"
)
print("✅ Model Initialized")

# --- Prompt to AI ---
prompt = f"""Check this assignment for plagiarism.
Compare it with the reference materials below.

User Submission:
{user_text}

Reference 1:
{ref1_text}

Reference 2:
{ref2_text}
"""

response = model.generate(prompt=prompt)

# --- Save & Show Report ---
try:
    if isinstance(response, str):
        response = json.loads(response)

    report = response['results'][0]['generated_text']
    print("\n🧠 Plagiarism Detection Report:\n")
    print(report)

    with open("plagiarism_report.txt", "w", encoding="utf-8") as f:
        f.write(report)

    print("✅ Report saved as plagiarism_report.txt")

except Exception as e:
    print("⚠️ Failed to parse response.")
    print("Raw output:\n", response)
    print("\nError:\n", e)


✅ Downloaded: user.txt
✅ Downloaded: ref1.txt
✅ Downloaded: ref2.txt


/opt/conda/envs/Python-RT24.1/lib/python3.11/site-packages/ibm_watsonx_ai/foundation_models/utils/utils.py:436: LifecycleWarning: Model 'ibm/granite-13b-instruct-v2' is in deprecated state from 2025-06-18 until 2025-10-15. IDs of alternative models: ibm/granite-3-3-8b-instruct. Further details: https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/fm-model-lifecycle.html?context=wx&audience=wdp
  warn(model_state_warning, category=LifecycleWarning)


✅ Model Initialized

🧠 Plagiarism Detection Report:


✅ Report saved as plagiarism_report.txt
